# Basic MNIST Example

Like 99% of all other ML tutorials, we start with the analysis of handwritten digits. The MNIST dataset contains 70,000 images (28x28 pixels in size) of handwritten digits from 0 to 9, and is the _de facto_ dataset for experimenting with classifiers and neural networks. The current state of the art achieves a classification accuracy of 99.82% (Kowsari _et al._ [2018](10.1145/3206098.3206111)), which might be limited by the labelling accuracy (humans make errors too).

In this tutorial, we will build a simple classifier in [Tensorflow](https://www.tensorflow.org/) that contains many of the basic ingredients needed in deep learning. As always, we start with importing the necessary libraries:

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

## Data preparation and inspection

Next, we load the MNIST dataset, which is conveniently provided by Keras

In [ ]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.shape, test_images.shape)

To get a sense of what this dataset looks like, we'll plot the first 50 numbers:

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=10)
# Loop over all rows
for i in range(5):
    # Loop over all columns
    for j in range(10):
        n = i*10 + j
        axes[i, j].set_title("%d" % train_labels[n])
        axes[i, j].imshow(train_images[n], cmap="gray")
        axes[i, j].axis("off")
plt.tight_layout()
plt.show()

As you can see, some digits are more clearly written than others. It is this diversity that makes it hard to get good performance on this dataset with traditional methods, as compared with machine learning.

Before we start training, we pre-process the dataset by scaling the pixel values between 0 and 1 (you can try for yourself to see the effect on the model performance by taking out this step).

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

## Model construction

Next, we set-up the model. To help us with the wiring, we use the Keras API. The model will consist of 3 operations: first we convert the 28x28 image into a 784-sized vector (`Flatten`), then we add two fully-connected (`Dense`) layers. The first dense layers has an output size of 128 with ReLU activation, the second layer has an output size of 10 (one for each digit from 0-9), with [softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) activation. (Note that the code below may raise a `WARNING`. You can ignore this)

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=train_images[0].shape),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

The softmax function takes the output vector, and scales all values such that they sum up to 1. In this way, we get a vector of probabilities. The first entry in the output corresponds to the probability that the input image is a 0, the second entry that the input is 1, etc.:

$$
P = \left[\begin{matrix} p(0) \\ p(1) \\ p(2) \\ ... \\ p(9) \end{matrix} \right] \quad , \quad \sum_{i=0}^9 P_i = 1
$$

We now have to choose a loss function. For multi-class classification tasks, _categorical cross-entropy_ is usually a good choice. This loss function is defined as follows:

$$
\mathcal{L} = - \sum_{c=0}^N y_c \log \left( p_c \right)
$$

where $y_c$ is the label of class $c$, and $p$ is the predicted probability. Note that $y_c$ is either 0 or 1, and that $0 < p_c < 1$. With our chosen loss function, we are ready for the final assembly of the model.

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
print(model.summary())

As we can see from the `model.summary()`, there are over 100,000 free parameters that need to be constrained. This may seem a little excessive, given that we have only 60,000 samples in the training set, but this is actually considered a rather "shallow" model; it is not uncommon to have millions of free parameters in production-level models.

We can verify ourselves that the numbers quoted above are correct: the first layer has an input size of 784 elements, and an output size of 128. Recall that $x_{n+1} = W_{ij} x_n + b_j$, so we have a weight matrix of size $784 \times 128 = 100,352$, and a bias vector of size $128$, which gives $100,352 + 128 = 100,480$ trainable parameters for the first layer. The second layer takes the output of the first layer as an input, so that we have $128 \times 10 = 1280$ weights and $10$ biases. This gives a grand total of $100,480 + 1290 = 101,770$ trainable parameters.

## Training

Now that everything has been set-up, we feed our training data into the model and start training:

In [ ]:
model.fit(
    train_images, 
    train_labels, 
    validation_data=(test_images, test_labels),
    verbose=1,
    epochs=5)

There are two things to notice in the training logs: 

1) The accuracy on the training set is higher than that of the validation test. This is quite commonly the case, and to be expected: the gradient-descent optimiser will try to mimise the loss calculated on the _training_ set, but not on the _validation_ set. In an extreme case of optimisation, the loss on the training set will be zero (= 100% accuracy), but that can only be achieved by "memorising" the training set (overfitting). Consequently, the accuracy on the validation/test set will be poor, because there is no way to memorise the validation/test data.

2) The values of the loss and accuracy will be different each time you run the training process. This is because of the random initialisation of the weights. Sometime you'll get slightly better performance because you happened to start the training process in a more favourable spot. This may seem trivial, but improper initialisation is what stalled the development of deep neural networks in the late 90's! (See [this webpage](https://www.deeplearning.ai/ai-notes/initialization/) for an excellent visualisation of this technical problem)

After the training process has finished, we can evaluate the performance of the model and get our final predictions for the test set.

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)
print("Test accuracy: %.4f" % test_acc)

The test accuracy is likely around 97%, which is embarrasingly bad when compared with the [state of the art](https://paperswithcode.com/sota/image-classification-on-mnist), but it's not too bad considering we only employed two layers of neurons. 

## Visualisation

To visualise the performance, plot the first 50 samples of the test set, and print out the labels / predictions:

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=10)
for i in range(5):
    for j in range(10):
        n = i*10 + j
        pred_num = np.argmax(predictions[n])
        if pred_num == test_labels[n]:
            colour = "g"
        else:
            colour = "r"
        axes[i, j].set_title("%d / %d" % (test_labels[n], pred_num), c=colour)
        axes[i, j].imshow(test_images[n], cmap="gray")
        axes[i, j].axis("off")
plt.tight_layout()
plt.show()

Because we have 97% accuracy, most of the predictions will be correct (green colours). But once in a while, some bad predictions will show up. These bad predictions are most likely on rather ambiguously-written digits.

# Exercise

Experiment with different model configurations (number of layers, neurons per layer, activation functions, number of epochs, etc.) to see if you can beat the score of 97%. An example of a different configuration is given below:

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=train_images[0].shape),
    keras.layers.Dense(64, activation=tf.nn.tanh),  # Hyperbolic tangent
    keras.layers.Dense(32, activation=tf.nn.elu),   # Exponential Linear Unit
    keras.layers.Dense(16, activation=tf.nn.selu),  # Scaled Exponential Linear Unit
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(
    train_images, 
    train_labels, 
    validation_data=(test_images, test_labels),
    verbose=1,
    epochs=10)